In [ ]:
%matplotlib widget
from ipywidgets import *
import numpy as np
from datetime import datetime
from enstools.da.support.instruments.satellites.track_aeolus import aeolus_like_coordinates
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cartopy.crs as ccrs

Create plots of the reference time 2020-11-24

In [ ]:
start_time = datetime(2020, 11, 24)
end_time = datetime(2020, 11, 24, 1, 31)

In [ ]:
line1 = None
line2 = None
fig = plt.figure(2)

def sat_plot(longitude, latitude):
    ax1 = fig.add_subplot(1, 2, 1, projection=ccrs.Robinson())
    ax1.set_global()
    ax1.stock_img()
    ax1.coastlines()
    return ax1.plot(longitude, latitude,"r", transform=ccrs.Geodetic())


def orbit_plot(x,y,z):
    halfpi, pi, twopi = [f*np.pi for f in (0.5, 1, 2)]
    degs, rads = 180/pi, pi/180
    re = 6378.
    theta = np.linspace(0, twopi, 201)
    cth, sth, zth = [f(theta) for f in (np.cos, np.sin, np.zeros_like)]
    lon0 = re*np.vstack((cth, zth, sth))
    lons = []
    for phi in rads*np.arange(0, 180, 15):
        cph, sph = [f(phi) for f in (np.cos, np.sin)]
        lon = np.vstack((lon0[0]*cph - lon0[1]*sph,
                         lon0[1]*cph + lon0[0]*sph,
                         lon0[2]) )
        lons.append(lon)

    lat0 = re*np.vstack((cth, sth, zth))
    lats = []

    for phi in rads*np.arange(-75, 90, 15):
        cph, sph = [f(phi) for f in (np.cos, np.sin)]
        lat = re*np.vstack((cth*cph, sth*cph, zth+sph))
        lats.append(lat)

    ax2  = fig.add_subplot(1, 2, 2, projection='3d')

    for xlo, ylo, zlo in lons:
        ax2.plot(xlo, ylo, zlo, '-k')
    for xla, yla, zla in lats:
        ax2.plot(xla, yla, zla, '-k')
    return ax2.plot(x, y, z, "r-")


def update(start_lon, offset):
    global line1, line2
    lon, lat, x, y, z = aeolus_like_coordinates(start_time, end_time, start_lon, offset, include_metric=True)
    if line1 is None:
        line1, = sat_plot(lon, lat)
        line2, = orbit_plot(x,y,z)
    else:
        line1.set_ydata(lat)
        line1.set_xdata(lon)
        line2.set_data_3d(x,y,z)

In [ ]:
interact(update, start_lon=(-180,180), offset=(-180,180))